In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta
import ccxt
import ta
from scipy import stats
from sklearn import preprocessing

import ccxt
import ta
from  pyearth import Earth
import plotly.graph_objects as go
import re
from tqdm import tqdm

from sklearn.ensemble import RandomForestClassifier

In [2]:
def get_node_mars(model):
    list_summary = model.summary().split('\n')
    new_list_summary = []
                
    for j in list_summary:
        if 'Yes' not in j:
            new_list_summary.append(j)

    basis_function_numbers = []

    for line in new_list_summary:
        match = re.search(r'h\(x0-([\d.]+)\)|h\(([\d.]+)-x0\)|h\(([\d.]+)\)\s+No', line)
        if match:
            basis_function_numbers.extend(match.group(1, 2, 3))

    basis_number = []
    for i in basis_function_numbers:
        if not i is None:
            basis_number.append(float(i))
    basis_number.sort()
    return basis_number

In [3]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        # print("Query successful", end='\r')
        return cursor.fetchall()
    except Error as err:
        print(f"Error: '{err}'")

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor()
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
        # print("Query successful", end='\r')
    except Error as err:
        print(f"Error: '{err}'")

In [4]:
pw = '27111998'
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [5]:
exchange_id = 'tokocrypto'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})
markets = exchange.load_markets()
market_list = list(markets.keys())
considered_market = []
for i in market_list:
    if '/USDT' in i or '/BTC' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

604
['1000SATS/USDT', '1INCH/BTC', '1INCH/USDT', 'AAVE/USDT', 'ACA/BTC', 'ACA/USDT', 'ACE/BTC', 'ACE/USDT', 'ACH/BTC', 'ACH/USDT', 'ACM/USDT', 'ADA/BTC', 'ADA/USDT', 'ADX/BTC', 'ADX/USDT', 'AERGO/USDT', 'AGIX/BTC', 'AGIX/USDT', 'AGLD/BTC', 'AGLD/USDT', 'AI/BTC', 'AI/USDT', 'AKRO/USDT', 'ALCX/BTC', 'ALCX/USDT', 'ALGO/BTC', 'ALGO/USDT', 'ALICE/BTC', 'ALICE/USDT', 'ALPACA/BTC', 'ALPACA/USDT', 'ALPHA/BTC', 'ALPHA/USDT', 'ALPINE/BTC', 'ALPINE/USDT', 'ALT/BTC', 'ALT/USDT', 'AMP/USDT', 'ANKR/BTC', 'ANKR/USDT', 'APE/BTC', 'APE/USDT', 'API3/BTC', 'API3/USDT', 'APT/BTC', 'APT/USDT', 'ARB/BTC', 'ARB/USDT', 'ARKM/BTC', 'ARKM/USDT', 'ARPA/BTC', 'ARPA/USDT', 'AR/BTC', 'AR/USDT', 'ASR/USDT', 'ASTR/USDT', 'AST/BTC', 'ATA/BTC', 'ATA/USDT', 'ATM/USDT', 'ATOM/BTC', 'ATOM/USDT', 'AUCTION/BTC', 'AUCTION/USDT', 'AUDIO/BTC', 'AUDIO/USDT', 'AVAX/BTC', 'AVAX/USDT', 'AVA/BTC', 'AVA/USDT', 'AXL/BTC', 'AXL/USDT', 'AXS/BTC', 'AXS/USDT', 'BADGER/BTC', 'BADGER/USDT', 'BAL/BTC', 'BAL/USDT', 'BAND/BTC', 'BAND/USDT', '

In [8]:
import warnings
warnings.filterwarnings('ignore')

arr_last_slope = []
arr_last_slope_adi = []
arr_aroon_up = []
arr_aroon_down = []
arr_aroon_diff = []
arr_macd_diff = []
arr_macd_line = []
arr_macd_signal = []
arr_stochastic = []
arr_stochastic_signal = []
arr_delta_stochastic = []
arr_rsi_val = []
arr_coef_corr = []
arr_last_slope_btc = []


arr_is_pump = []

arr_market = []
arr_start_date = []

WINDOW_AFTER = 14
for idx in tqdm(range(len(considered_market))):
    try:
        market = considered_market[idx]
        market = market.replace('/','')
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        if max(df['timestamp']) < 1709251200000:
            raise Exception("%s Database not up to date" % market)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        # df = df[df['timestamp'] <= start_pump]
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df['timestamp'] = day_df.index

        # start_point = random.choice(day_df['timestamp'])

        # current_price = day_df.loc[start_point]['close']
        
        # df_temp_next = day_df[day_df['timestamp'] > start_point]
        # df_temp_next = df_temp_next[df_temp_next['timestamp'] < start_point + pd.DateOffset(days=WINDOW_AFTER)]
        # max_price = max(df_temp_next['close'])

        # if (max_price - current_price)/current_price > 0.5:
        #     print(market, start_point)
        #     print((max_price - current_price)/current_price)
        #     # continue

        # df = df[df['timestamp'] <= start_point]
        # day_df = day_df[day_df['timestamp'] <= start_point]
        num_idx = min([365, len(day_df)])

        model_price = Earth()
        scaler = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price = scaler.fit_transform(day_df['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_price.fit(arr_x,scaled_price.flatten())
        y_pred = model_price.predict(arr_x)
        basis_node = get_node_mars(model_price)
        last_node = int(basis_node[-1])
        last_slope = (y_pred[-1] - y_pred[last_node])/(arr_x[-1] - arr_x[last_node])

        adi = ta.volume.AccDistIndexIndicator(high=df['high'],low=df['low'],close=df['close'],volume=df['volume']).acc_dist_index()
        model_adi = Earth()
        scaler_adi = preprocessing.MinMaxScaler(feature_range=(0,1000))
        data_num_adi = min([2000,len(df)])
        scaled_adi = scaler_adi.fit_transform(df['close'].iloc[-data_num_adi:].values.reshape(-1,1))
        arr_x_adi = np.arange(data_num_adi)/24
        model_adi.fit(arr_x_adi,scaled_adi.flatten())

        y_pred_adi = model_adi.predict(arr_x_adi)
        basis_node = get_node_mars(model_adi)
        last_node = basis_node[-1]
        last_slope_adi = (y_pred_adi[-1] - model_adi.predict([last_node]))[0]/(arr_x[-1] - last_node)

        aroon = ta.trend.AroonIndicator(low=day_df['low'],high=day_df['high'])
        aroon_up = aroon.aroon_up()[-1]
        aroon_down = aroon.aroon_down()[-1]
        aroon_diff = aroon.aroon_indicator()[-1]

        macd = ta.trend.MACD(close=pd.Series(scaled_price.flatten()))
        macd_diff = macd.macd_diff().values[-1]
        macd_line = macd.macd().values[-1]
        macd_signal = macd.macd_signal().values[-1]

        stoch_oscillator = ta.momentum.StochasticOscillator(high=day_df['high'], low=day_df['low'], close=day_df['close'])
        stochastic = stoch_oscillator.stoch().values[-1]
        stochastic_signal = stoch_oscillator.stoch_signal().values[-1]
        delta_stochastic = stochastic - stochastic_signal

        rsi_indicator = ta.momentum.RSIIndicator(close=pd.Series(scaled_price.flatten()))
        rsi_val = rsi_indicator.rsi().values[-1]

        query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
        res = execute_query(connection,query)
        df_btc = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df_btc = df_btc.drop_duplicates()
        df_btc['timestamp'] = pd.to_datetime(df_btc['timestamp'], unit='ms')
        # df_btc = df_btc[df_btc['timestamp'] <= start_point]
        
        day_df_btc = (df_btc.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'mean'}))
        day_df_btc['timestamp'] = day_df_btc.index

        alt_vs_btc = pd.DataFrame({'alt': day_df['close'].iloc[-num_idx:], 'btc': day_df_btc['close'].iloc[-num_idx:]})
        df_corr = alt_vs_btc.corr()
        coef_corr = df_corr['alt']['btc']

        model_btc = Earth()
        scaler_btc = preprocessing.MinMaxScaler(feature_range=(0,100))
        scaled_price_btc = scaler_btc.fit_transform(day_df_btc['close'].iloc[-num_idx:].values.reshape(-1,1))

        arr_x = np.arange(num_idx)
        model_btc.fit(arr_x,scaled_price_btc.flatten())

        y_pred_btc = model_btc.predict(arr_x)
        basis_node_btc = get_node_mars(model_btc)
        last_node_btc = int(basis_node_btc[-1])
        last_slope_btc = (y_pred_btc[-1] - y_pred_btc[last_node_btc])/(arr_x[-1] - arr_x[last_node_btc])

        arr_last_slope.append(last_slope)
        arr_last_slope_adi.append(last_slope_adi)
        arr_aroon_up.append(aroon_up)
        arr_aroon_down.append(aroon_down)
        arr_aroon_diff.append(aroon_diff)
        arr_macd_diff.append(macd_diff)
        arr_macd_line.append(macd_line)
        arr_macd_signal.append(macd_signal)
        arr_stochastic.append(stochastic)
        arr_stochastic_signal.append(stochastic_signal)
        arr_delta_stochastic.append(delta_stochastic)
        arr_rsi_val.append(rsi_val)
        arr_coef_corr.append(coef_corr)
        arr_last_slope_btc.append(last_slope_btc)
        arr_market.append(market)
        # arr_start_date.append(start_point)
    except Exception as e:
        print('Error: ', e)
        arr_last_slope.append(np.nan)
        arr_last_slope_adi.append(np.nan)
        arr_aroon_up.append(np.nan)
        arr_aroon_down.append(np.nan)
        arr_aroon_diff.append(np.nan)
        arr_macd_diff.append(np.nan)
        arr_macd_line.append(np.nan)
        arr_macd_signal.append(np.nan)
        arr_stochastic.append(np.nan)
        arr_stochastic_signal.append(np.nan)
        arr_delta_stochastic.append(np.nan)
        arr_rsi_val.append(np.nan)
        arr_coef_corr.append(np.nan)
        arr_last_slope_btc.append(np.nan)
        arr_market.append(market)
        # arr_start_date.append(start_point)
        continue

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 2/604 [00:00<00:43, 13.68it/s]

Error: '1146 (42S02): Table 'crypto_db.1000satsusdt' doesn't exist'
Error:  max() arg is an empty sequence
Error:  1INCHBTC Database not up to date


  1%|          | 5/604 [00:24<50:03,  5.01s/it]  

Error:  ACABTC Database not up to date


...

100%|█████████▉| 601/604 [34:51<00:11,  3.82s/it]

Error:  ZILBTC Database not up to date


100%|█████████▉| 603/604 [34:56<00:02,  2.88s/it]

Error:  ZRXBTC Database not up to date


100%|██████████| 604/604 [35:00<00:00,  3.48s/it]


In [9]:
current_data = pd.DataFrame({'market': arr_market,
                         })
current_data['last_slope'] = arr_last_slope
current_data['last_slope_adi'] = arr_last_slope_adi
current_data['aroon_up'] = arr_aroon_up
current_data['aroon_down'] = arr_aroon_down
current_data['aroon_diff'] = arr_aroon_diff
current_data['macd_diff'] = arr_macd_diff
current_data['macd_line'] = arr_macd_line
current_data['macd_signal'] = arr_macd_signal
current_data['stochastic'] = arr_stochastic
current_data['stochastic_signal'] = arr_stochastic_signal
current_data['delta_stochastic'] = arr_delta_stochastic
current_data['rsi_val'] = arr_rsi_val
current_data['coef_corr'] = arr_coef_corr
current_data['last_slope_btc'] = arr_last_slope_btc

In [13]:
current_data = current_data.dropna()

In [14]:
current_data.head()

,market,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,stochastic_signal,delta_stochastic,rsi_val,coef_corr,last_slope_btc
2,1INCHUSDT,3.106811,0.306093,96.0,0.0,96.0,2.591268,11.604379,9.013111,86.252546,91.952381,-5.699835,70.735196,0.448768,1.150705
3,AAVEUSDT,2.730850,0.597512,100.0,0.0,100.0,3.250811,10.306344,7.055534,82.498268,93.662382,-11.164114,70.324743,0.857666,1.150705
5,ACAUSDT,2.473462,0.455137,80.0,36.0,44.0,1.352613,7.175834,5.823221,59.198543,58.545633,0.652910,62.492731,0.652707,1.150705
7,ACEUSDT,1.888034,101.530333,84.0,0.0,84.0,0.845454,10.509408,9.663954,70.794398,68.671429,2.122969,60.795865,0.499572,1.764322
9,ACHUSDT,1.486491,-0.241926,96.0,0.0,96.0,1.435127,9.022757,7.587630,80.590717,76.030109,4.560608,71.306800,0.018787,1.150705


In [15]:
df_data_pump = pd.read_csv('data_pump_not_ico.csv')
df_data_not_pump = pd.read_csv('data_not_pump.csv')

In [16]:
df_pump_cand = df_data_pump[['market','start','last_slope', 'last_slope_adi',
       'aroon_up', 'aroon_down', 'aroon_diff', 'macd_diff', 'macd_line',
       'macd_signal', 'stochastic', 'stochastic_signal', 'delta_stochastic',
       'rsi_val', 'coef_corr', 'last_slope_btc']]
df_not_pump_cand = df_data_not_pump[['market','start','last_slope', 'last_slope_adi',
       'aroon_up', 'aroon_down', 'aroon_diff', 'macd_diff', 'macd_line',
       'macd_signal', 'stochastic', 'stochastic_signal', 'delta_stochastic',
       'rsi_val', 'coef_corr', 'last_slope_btc']]

In [17]:
df_not_pump_cand['is_pump'] = np.zeros(len(df_not_pump_cand))
df_pump_cand['is_pump'] = np.ones(len(df_pump_cand))

In [18]:
df_combine = pd.concat([df_not_pump_cand,df_pump_cand])

In [19]:
arr_prob_last_slope = []
arr_prob_macd = []
arr_prob_macd_diff = []
arr_prob_stochastic = []
arr_prob_rsi = []
arr_prob_corr_btc = []

weight_data_pump = len(df_data_not_pump)/len(df_data_pump)
print(weight_data_pump)

for i in tqdm(range(len(current_data))):
    var = current_data.iloc[i]
    last_slope_val = var['last_slope']
    band = 0.1
    last_slope = {}
    num_pump = weight_data_pump*len(df_data_pump.loc[(df_data_pump['last_slope'] >= last_slope_val - band) & (df_data_pump['last_slope'] <= last_slope_val + band)])
    num_not_pump = len(df_data_not_pump.loc[(df_data_not_pump['last_slope'] >= last_slope_val - band) & (df_data_not_pump['last_slope'] <= last_slope_val + band)])
    if num_pump+num_not_pump == 0:
        last_slope['1'] = 0.5
        last_slope['0'] = 1 - last_slope['1']
    else:
        last_slope['1'] = num_pump/(num_pump+num_not_pump)
        last_slope['0'] = 1 - last_slope['1']
    
    macd_line_val = var['macd_line']
    macd_signal_val = var['macd_signal']
    band_line = 0.5
    band_signal = 0.5
    macd_line_signal = {}
    temp = df_combine.loc[(df_combine['macd_signal'] >= macd_signal_val - band_signal) &
                        (df_combine['macd_signal'] <= macd_signal_val + band_signal) &
                        (df_combine['macd_line'] >= macd_line_val - band_line) &
                        (df_combine['macd_line'] <= macd_line_val + band_line)]
    num_pump = weight_data_pump*sum(temp['is_pump'])
    num_not_pump = len(temp['is_pump'])-sum(temp['is_pump'])
    if num_pump+num_not_pump == 0:
        macd_line_signal['1'] = 0.5
        macd_line_signal['0'] = 1 - macd_line_signal['1']
    else:
        macd_line_signal['1'] = num_pump/(num_pump+num_not_pump)
        macd_line_signal['0'] = 1 - macd_line_signal['1']
    
    macd_diff_val = var['macd_diff']
    band = 0.1
    macd_diff = {}
    num_pump = weight_data_pump*len(df_data_pump.loc[(df_data_pump['macd_diff'] >= macd_diff_val - band) & (df_data_pump['macd_diff'] <= macd_diff_val + band)])
    num_not_pump = len(df_data_not_pump.loc[(df_data_not_pump['macd_diff'] >= macd_diff_val - band) & (df_data_not_pump['macd_diff'] <= macd_diff_val + band)])
    if num_pump+num_not_pump == 0:
        macd_diff['1'] = 0.5
        macd_diff['0'] = 1 - macd_diff['1']
    else:
        macd_diff['1'] = num_pump/(num_pump+num_not_pump)
        macd_diff['0'] = 1 - macd_diff['1']
    
    stochastic_val = var['stochastic']
    stochastic_signal_val = var['stochastic_signal']
    band_line = 2.5
    band_signal = 2.5
    stochastic_line_signal = {}
    temp = df_combine.loc[(df_combine['stochastic'] >= stochastic_val - band_line) &
                        (df_combine['stochastic'] <= stochastic_val + band_line) &
                        (df_combine['stochastic_signal'] >= stochastic_signal_val - band_signal) &
                        (df_combine['stochastic_signal'] <= stochastic_signal_val + band_signal)]
    num_pump = weight_data_pump*sum(temp['is_pump'])
    num_not_pump = len(temp['is_pump'])-sum(temp['is_pump'])
    if num_pump+num_not_pump == 0:
        stochastic_line_signal['1'] = 0.5
        stochastic_line_signal['0'] = 1 - stochastic_line_signal['1']
    else:
        stochastic_line_signal['1'] = num_pump/(num_pump+num_not_pump)
        stochastic_line_signal['0'] = 1 - stochastic_line_signal['1']
    
    rsi_val = var['rsi_val']
    band = 2.5
    rsi = {}
    num_pump = weight_data_pump*len(df_data_pump.loc[(df_data_pump['rsi_val'] >= rsi_val - band) & (df_data_pump['rsi_val'] <= rsi_val + band)])
    num_not_pump = len(df_data_not_pump.loc[(df_data_not_pump['rsi_val'] >= rsi_val - band) & (df_data_not_pump['rsi_val'] <= rsi_val + band)])
    if num_pump+num_not_pump == 0:
        rsi['1'] = 0.5
        rsi['0'] = 1 - rsi['1']
    else:
        rsi['1'] = num_pump/(num_pump+num_not_pump)
        rsi['0'] = 1 - rsi['1']
    
    coef_corr_val = var['coef_corr']
    slope_btc_val = var['last_slope_btc']
    band_corr = 0.05
    band_slope = 0.25
    corr_slope_btc = {}
    temp = df_combine.loc[(df_combine['coef_corr'] >= coef_corr_val - band_corr) &
                        (df_combine['coef_corr'] <= coef_corr_val + band_corr) &
                        (df_combine['last_slope_btc'] >= slope_btc_val - band_slope) &
                        (df_combine['last_slope_btc'] <= slope_btc_val + band_slope)]
    num_pump = weight_data_pump*sum(temp['is_pump'])
    num_not_pump = len(temp['is_pump'])-sum(temp['is_pump'])
    if num_pump+num_not_pump == 0:
        corr_slope_btc['1'] = 0.5
        corr_slope_btc['0'] = 0.5
    else:
        corr_slope_btc['1'] = num_pump/(num_pump+num_not_pump)
        corr_slope_btc['0'] = 1 - corr_slope_btc['1']
    
    arr_prob_last_slope.append(last_slope['1'])
    arr_prob_macd.append(macd_line_signal['1'])
    arr_prob_macd_diff.append(macd_diff['1'])
    arr_prob_stochastic.append(stochastic_line_signal['1'])
    arr_prob_rsi.append(rsi['1'])
    arr_prob_corr_btc.append(corr_slope_btc['1'])


1.7243532560214094


100%|██████████| 327/327 [00:01<00:00, 167.04it/s]


In [20]:
current_data['prob_last_slope'] = arr_prob_last_slope
current_data['prob_macd'] = arr_prob_macd
current_data['prob_macd_diff'] = arr_prob_macd_diff
current_data['prob_stochastic'] = arr_prob_stochastic
current_data['prob_rsi'] = arr_prob_rsi
current_data['prob_corr_btc'] = arr_prob_corr_btc
current_data.head()

,market,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,...,delta_stochastic,rsi_val,coef_corr,last_slope_btc,prob_last_slope,prob_macd,prob_macd_diff,prob_stochastic,prob_rsi,prob_corr_btc
2,1INCHUSDT,3.106811,0.306093,96.0,0.0,96.0,2.591268,11.604379,9.013111,86.252546,...,-5.699835,70.735196,0.448768,1.150705,0.563940,0.000000,0.223236,0.364992,0.220070,0.579741
3,AAVEUSDT,2.730850,0.597512,100.0,0.0,100.0,3.250811,10.306344,7.055534,82.498268,...,-11.164114,70.324743,0.857666,1.150705,0.632940,0.000000,0.462994,0.000000,0.284655,0.481462
5,ACAUSDT,2.473462,0.455137,80.0,36.0,44.0,1.352613,7.175834,5.823221,59.198543,...,0.652910,62.492731,0.652707,1.150705,0.340937,0.462994,0.230703,0.632940,0.427023,0.508506
7,ACEUSDT,1.888034,101.530333,84.0,0.0,84.0,0.845454,10.509408,9.663954,70.794398,...,2.122969,60.795865,0.499572,1.764322,0.064524,1.000000,0.310670,0.392700,0.462994,0.741864
9,ACHUSDT,1.486491,-0.241926,96.0,0.0,96.0,1.435127,9.022757,7.587630,80.590717,...,4.560608,71.306800,0.018787,1.150705,0.492373,0.462994,0.201498,0.209664,0.209664,0.197648


In [21]:
current_data.to_csv('current_data_early_mar.csv')

In [22]:
import pickle
import joblib

logreg_model = pickle.load(open('logreg_model.sav', 'rb'))
# rf_model = joblib.load("rf_model.joblib")
# rf_model = pickle.load(open('rf_model.sav', 'rb'))

In [23]:
X = current_data[['prob_last_slope','prob_macd','prob_macd_diff','prob_stochastic','prob_rsi','prob_corr_btc']]
X

,prob_last_slope,prob_macd,prob_macd_diff,prob_stochastic,prob_rsi,prob_corr_btc
2,0.563940,0.000000,0.223236,0.364992,0.220070,0.579741
3,0.632940,0.000000,0.462994,0.000000,0.284655,0.481462
5,0.340937,0.462994,0.230703,0.632940,0.427023,0.508506
7,0.064524,1.000000,0.310670,0.392700,0.462994,0.741864
9,0.492373,0.462994,0.201498,0.209664,0.209664,0.197648
...,...,...,...,...,...,...
595,0.256434,0.000000,0.312137,0.439398,0.000000,0.534790
597,0.632940,0.838006,0.472108,0.291062,0.490563,0.596452
599,0.433870,1.000000,0.456281,0.392700,0.384124,0.579741
601,0.000000,1.000000,0.000000,0.496309,0.220070,0.496309


In [24]:
pred = logreg_model.predict(X)
current_data['prediction'] = pred

In [25]:
probability = logreg_model.predict_proba(X)
current_data['probability'] = probability[:,1]
current_data.head()

,market,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,...,coef_corr,last_slope_btc,prob_last_slope,prob_macd,prob_macd_diff,prob_stochastic,prob_rsi,prob_corr_btc,prediction,probability
2,1INCHUSDT,3.106811,0.306093,96.0,0.0,96.0,2.591268,11.604379,9.013111,86.252546,...,0.448768,1.150705,0.563940,0.000000,0.223236,0.364992,0.220070,0.579741,0.0,0.061564
3,AAVEUSDT,2.730850,0.597512,100.0,0.0,100.0,3.250811,10.306344,7.055534,82.498268,...,0.857666,1.150705,0.632940,0.000000,0.462994,0.000000,0.284655,0.481462,0.0,0.009924
5,ACAUSDT,2.473462,0.455137,80.0,36.0,44.0,1.352613,7.175834,5.823221,59.198543,...,0.652707,1.150705,0.340937,0.462994,0.230703,0.632940,0.427023,0.508506,1.0,0.506185
7,ACEUSDT,1.888034,101.530333,84.0,0.0,84.0,0.845454,10.509408,9.663954,70.794398,...,0.499572,1.764322,0.064524,1.000000,0.310670,0.392700,0.462994,0.741864,1.0,0.872497
9,ACHUSDT,1.486491,-0.241926,96.0,0.0,96.0,1.435127,9.022757,7.587630,80.590717,...,0.018787,1.150705,0.492373,0.462994,0.201498,0.209664,0.209664,0.197648,0.0,0.025809


In [44]:
sorted_data = current_data.sort_values(by=['probability'],ascending=False)
sorted_data.head()

,market,last_slope,last_slope_adi,aroon_up,aroon_down,aroon_diff,macd_diff,macd_line,macd_signal,stochastic,...,coef_corr,last_slope_btc,prob_last_slope,prob_macd,prob_macd_diff,prob_stochastic,prob_rsi,prob_corr_btc,prediction,probability
594,XVSBTC,1.142382,0.483290,16.0,80.0,-64.0,0.033693,-6.325839,-6.359532,61.233480,...,0.572239,2.346307,0.472047,0.596452,0.466356,1.000000,0.485183,0.775216,1.0,0.982096
406,PENDLEBTC,-1.804660,-1.039585,52.0,0.0,52.0,-3.630048,3.559793,7.189840,11.755878,...,0.661198,3.063709,0.548979,0.838006,0.873376,0.547861,0.512685,0.873376,1.0,0.976217
174,DUSKBTC,-1.154110,-0.741820,52.0,0.0,52.0,-2.798122,2.528796,5.326918,3.553299,...,-0.229493,2.346307,0.558406,0.696890,0.523200,0.744777,0.469156,0.838006,1.0,0.973258
261,HIVEBTC,-0.218636,-1.156600,4.0,88.0,-84.0,-0.059545,-2.184918,-2.125373,22.772277,...,-0.789102,2.346307,0.489084,0.537479,0.465693,0.639282,0.676779,1.000000,1.0,0.966599
541,UFTUSDT,1.438490,-0.422306,76.0,0.0,76.0,-0.182377,4.207506,4.389883,37.619048,...,0.453694,2.779080,0.479073,0.775216,0.522665,0.632940,0.538162,0.838006,1.0,0.966354


In [45]:
sorted_data.to_csv('logreg_model.csv')

In [28]:
current_data.to_csv('logreg_model_not_sorted_early_mar.csv')

In [27]:
# joblib.dump(rf, "my_random_forest.joblib")

# load
loaded_rf = joblib.load("rf_model.joblib")


ValueError: Did not recognise loaded array layout